# Adding AgentOps to an existing Autogen service
<img src="https://raw.githubusercontent.com/AgentOps-AI/agentops/main/docs/images/external/autogen/autogen_integration.png?raw=true"/>

To get started, you'll need to install the AgentOps package and [set an API key](app.agentops.ai).

AgentOps automatically configures itself when it's initialized meaning your agent run data will be tracked and logged to your AgentOps account right away.

In [1]:
from typing import Annotated, Literal
from autogen import ConversableAgent, register_function
import agentops

agentops.init(api_key="...", tags=["autogen-tool-example"])

print("AgentOps is now running. You can view your session in the link above")

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=51556b00-428b-4cae-b815-1320ebc7e810


AgentOps is now running. You can view your session in the link above


AutoGen will now start automatically tracking

* LLM prompts and completions
* Token usage and costs
* Agent names and actions
* Correspondence between agents
* Tool usage
* Errors

In [2]:
# Define an openai api key for the agent configurations
openai_api_key = "..."
config_list = [
    {
        "model": "gpt-4-turbo",
        "api_key": openai_api_key,
        "tags": ["gpt-4", "tool"]
    }
]

In [3]:
Operator = Literal["+", "-", "*", "/"]

def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

# Create the agent that uses the LLM.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)
user_proxy.register_for_execution(name="calculator")(calculator)

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Let the assistant start the conversation.  It will end when the user types "exit".
user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

agentops.end_session("Success")

🖇 AgentOps: Cannot start session - session already started


User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...


/Users/reibs/Projects/autogen/autogen/agentchat/conversable_agent.py:2489: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{tool_sig['function']['name']}' is being overridden.", UserWarning)
/Users/reibs/Projects/autogen/autogen/agentchat/conversable_agent.py:2408: UserWarning: Function 'calculator' is being overridden.
  warnings.warn(f"Function '{name}' is being overridden.", UserWarning)


Assistant (to User):

***** Suggested tool call (call_pSJXJKu1qbfRV4SVNHzTaG1z): calculator *****
Arguments: 
{"a": 1423, "b": 123, "operator": "-"}
***************************************************************************
***** Suggested tool call (call_kHRXi8vq5XsZSrGDnVTA1oy7): calculator *****
Arguments: 
{"a": 32, "b": 23, "operator": "+"}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_pSJXJKu1qbfRV4SVNHzTaG1z) *****
1300
**********************************************************************

--------------------------------------------------------------------------------
User (to Assistant):

***** Response from calling tool (call_kHRXi8vq5XsZSrGDnVTA1oy7) *****
55
**************************************

🖇 AgentOps: This run's cost $0.000600
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=51556b00-428b-4cae-b815-1320ebc7e810


You can see your run in action at [app.agentops.ai](app.agentops.ai). In this example, the AgentOps dashboard will show:

* Agents talking to each other
* Each use of the calculator tool
* Each call to OpenAI for LLM use